Algorithm 1(eliminate points-m): 
    Input: p1,p2,... , pn' (in order) where n' is the number of points in the stream. 
    Output: Skyline points S' 
    1: Let x = 24m. 
    2: Pass 1: For j : 1, 2, ..., x, let p'j be a point picked uniformly at random from the stream. 
    Let S be the set of such points.
    3: Pass 2: 
    4: for i = 1, ..., n' do 
    5: for any p'j, if pi dominates p'j then p'j:=pi
    6: end for 
    7: Let S'={p'1,p'2,...,p'x}.
    8: Pass 3: 
            Delete from stream all points in S' and all points dominated by any point in S'.
    9: return S' 

In [36]:
from random import randint

class algorithm1:
    def __init__(self,amount_skylinepoints, dimension, stream_size): 
        self._amount_skylinepoints = amount_skylinepoints
        self._dimension = dimension 
        self._stream_size = n
    
    def transform_stream_tuple_in_list(stream):
        stream_list= []     
        for component in stream:
            stream_list.append(component)
        return(stream_list)
    
    def stream_creation(_dimension,_stream_size):
        stream = ()
        for point in range(_stream_size):
            coordinates = ()
            for coordinate in range(_dimension):
                coordinates = coordinates + (randint(1,100),)
            stream = stream +(coordinates,)
        return(algorithm1.transform_stream_tuple_in_list(stream))

    def reservoir_sampling(stream,_amount_skylinepoints):
        if 24*_amount_skylinepoints>(len(stream)-1):
            print('there are too many skylinepoints')
        else: 
            S = []
            size_sample = 24*_amount_skylinepoints
            for initial_sample in range(size_sample):
                S.append(stream[initial_sample])            

            for address_candidate_sample, candidate_sample in enumerate(stream): 
                chance = randint(0,address_candidate_sample)
                if chance < size_sample:
                    S[chance] = candidate_sample
        
        return(S) 
    
    def random_domination_from_stream_to_reservoir(stream, S):  
        for candidate in stream: 
            chance = randint(0,len(S)-1)
            if candidate[0] > S[chance][0] and candidate[1] > S[chance][1]:
                S[chance] = candidate
        return(S)

    def elimination_S_components_from_stream(stream,S):
        for proponent in S:
            for opponent in stream:
                if opponent == proponent:
                    stream.remove(opponent)
        return(stream)
    
    # the previous function is of no use if we use => instead of > here: when I tested I didn't see any real difference
    def elimination_dominated_by_S_components_from_stream(stream,S):
        for adress_component, component in enumerate(S):
            for adress_other_component, other_component in enumerate(S):
                if component == other_component and adress_component != adress_other_component:
                    S.remove(other_component)
        for candidate in S:
                for victim in stream: 
                    if victim in stream and candidate>victim:
                            stream.remove(victim)
        return(stream,S)           

stream_before_purge = algorithm1.stream_creation(2,10000)
print(('stream_before_purge'))
print(len(stream_before_purge))
reservoir_before_random_domination = algorithm1.reservoir_sampling(stream_before_purge,100)
reservoir_after_random_domination = algorithm1.random_domination_from_stream_to_reservoir(stream_before_purge,reservoir_before_random_domination)
print(('reservoir_after_random_domination'))
print(len(reservoir_after_random_domination))
stream_after_purge_reservoir = algorithm1.elimination_S_components_from_stream(stream_before_purge,reservoir_after_random_domination)
print(('stream_after_purge_reservoir'))
print(len(stream_after_purge_reservoir))
stream_after_domination = algorithm1.elimination_dominated_by_S_components_from_stream(stream_after_purge_reservoir,reservoir_after_random_domination)
print(('stream_after_domination'))
print(len(stream_after_domination[0]))
print(('reservoir after removal duplicates'))
print(len(stream_after_domination[1]))

stream_before_purge
10000
reservoir_after_random_domination
2400
stream_after_purge_reservoir
6728
stream_after_domination
1
reservoir after removal duplicates
1721


Algorithm 2 (Streaming RAND): 
    1: Let n be the number of points in the input stream. 
    Let m' = 1. 
    2: while the input stream is not empty do: 
    3: let n' be the current number of points in the stream 
    4: Call eliminate points (m'log(nlogn))
    5: If more than n'/2 points are left in the stream, m' = 2 m'
    6: end while 
    Remark: In case the stream cannot be changed, we do not have to actually delete points from stream. 
    We only keep the skyline points found so far and consider only points in the stream that is not dominated by any found skyline points. 
        

In [6]:
''' Archives 
from random import randint 

class algorithm1:
    def __init__(self,skylinepoints,sizeS,dimension):
        self._skylinepoints = skylinepoints 
        self._sizeS = sizeS 
        self._dimension = dimension 
    
    def randomsample_creation(skylinepoints,dimension):
        randomsample = () 
        size_sample=skylinepoints*24 
        for sample in range(size_sample): 
            coordinates =() 
            for coordinate in range(self._dimension): 
                coordinates = coordinates+(randint(1,100),) 
            randomsample = randomsample + (coordinates,) 
        return(randomsample) 
    
    # randomsample selection 
    
    def list_of_tuples_creation(randomsample, dimension): 
        list_of_tuples = []
        for sample in randomsample:
            list_of_tuples.append(sample)
        return(list_of_tuples)
    # it's okay to have duplicates so delete this and adapt the rest 
    
    def creation_of_S(list_of_tuples,sizeS): 
        S = ()
        for sample in range(0,sizeS):
            newinput = list_of_tuples[randint(0,len(list_of_tuples)-1)] 
            S = S + (newinput,) 
            list_of_tuples.remove(newinput) # delete 
        return(S) 

    def creation_of_S_champion(Slist,dimension): 
        champion = ()
        origins = 0 
        while origins < dimension:
            champion = champion +(0,)
            origins += 1

        for pj in Slist:
            score = 0 
            while score < dimension:
                if pj[score]>champion[score]:
                    score += 1
                else:
                    break
            if score == dimension:
                champion = pj
        return(champion)

    def creation_of_skyline(skylinepoints,sizeS,dimension):
        skyline = () 
        randomsample_tuple = algorithm1.randomsample_creation(skylinepoints,dimension) 
        randomsample_list = algorithm1.list_of_tuples_creation(randomsample_tuple, dimension) 
        for skylinepoint in range(skylinepoints):
            Slist = algorithm1.creation_of_S(randomsample_list,sizeS)
            champion = algorithm1.creation_of_S_champion(Slist,dimension)
            skyline = skyline + (champion,)
        return(skyline,randomsample_tuple)

randomsampletuple = algorithm1.randomsample_creation(2,2)
randomsamplelist = algorithm1.list_of_tuples_creation(randomsampletuple,2)
first_S=algorithm1.creation_of_S(randomsamplelist,12)
champion = algorithm1.creation_of_S_champion(first_S)

result = algorithm1.creation_of_skyline(4,24,3)
print(result[0], 'is the supposed skyline from', result[1])
print(len(result[1]))
print(len(result[0])) '''

" Archives \nfrom random import randint \n\nclass algorithm1:\n    def __init__(self,skylinepoints,sizeS,dimension):\n        self._skylinepoints = skylinepoints \n        self._sizeS = sizeS \n        self._dimension = dimension \n    \n    def randomsample_creation(skylinepoints,dimension):\n        randomsample = () \n        size_sample=skylinepoints*24 \n        for sample in range(size_sample): \n            coordinates =() \n            for coordinate in range(self._dimension): \n                coordinates = coordinates+(randint(1,100),) \n            randomsample = randomsample + (coordinates,) \n        return(randomsample) \n    \n    # randomsample selection \n    \n    def list_of_tuples_creation(randomsample, dimension): \n        list_of_tuples = []\n        for sample in randomsample:\n            list_of_tuples.append(sample)\n        return(list_of_tuples)\n    # it's okay to have duplicates so delete this and adapt the rest \n    \n    def creation_of_S(list_of_tuple